# Machine learning
Tutta la parte dedicata al machine learning (regressione + classificazione) verrà scritta qui

In [7]:
import numpy as np
import geopandas as gpd
import pandas as pd

#sklearn
from sklearn.preprocessing import OneHotEncoder

# custom lib
import make_dataset as m_d
import sys
sys.path.append('./../src')

import ML_functions as supp_ML

np.random.seed(86122330)

# Prima task: prevedere l'attività di twitter a livello provinciale

In [74]:
# Importo i dati dal database finale (vedi Import Dati)
data = pd.read_csv(m_d.data_path_out / 'MachineLearningDB.csv')

# Smisto target e features
target_mattina = data['TargetDay']
target_sera = data['TargetNight']
data.drop(columns=['TargetDay', 'TargetNight'], inplace=True)

# Encoder sul dato categorico weekday
enc = OneHotEncoder(sparse=False, handle_unknown='ignore')
data["Weekday"]=enc.fit_transform(data["Weekday"].values[:,None]).tolist()


"""
NOTA: se inserisco un np.array(np.array) ad un dataframe MI TIENE SOLO PRIMO ELEMENTO (1.0<=>'Mo')
 Se converto a lista funziona
 
Nota: equivale a
data['Weekday'] = enc.fit(data["Weekday"].values[:,None])
data['Weekday'] = enc.transform(data["Weekday"].values[:,None]).tolist()

Nota: Per convertire i contenuti a np.array(), ma fcom
temp=enc.fit_transform(data["Weekday"].values[:,None])
data["Weekday"]=[np.array(i) for i in temp]
"""

#Restringere database di features, molte delle features ipotizzate contano poco, ed ho molti pochi dati
data=data[["Tweet1m", "Tweet2m", "Tweet1n", "Tweet2n", "Tavg1m", "Tavg2m", "Tavg1n", "Tavg2n", "Weekday"]]
data

[0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0]

In [72]:
#MATTINA:
supp_ML.logistic_regressor_fittato(data, target_mattina)
supp_ML.Random_Forest_Regressor_CV(data, target_mattina)


ValueError: setting an array element with a sequence.

# Osservazioni:
1) Altissima varianza valore r2, ho molti pochi dati per fare questa stima (also, ci sono i dati del fine anno e delle feste che sballano tutto) \
2) Rimuovere features aiuta abbastanza \
3) Includere il weekday **sembra** peggiorare la stima (probabilmente dovuto al fatto che finchè non lo fixiamo il one-hot-encoder hitta alcune colonne di numeri interi

Overall sembra che il predittore sia better-than random usando come features solo numero di tweets dei giorni precendenti, ma si trova comunque lontano da un buon predittore

In [4]:
# Random Forest Regressor
supp_ML.Random_Forest_Regressor_CV(data, target_mattina)
supp_ML.Random_Forest_Regressor_CV(data, target_sera)



Random forest r2_score =  0.12764175814188095
Fitting 3 folds for each of 432 candidates, totalling 1296 fits
Random forest r2_score =  0.13423090662168358
Random forest r2_score =  0.02151156262086995
Fitting 3 folds for each of 432 candidates, totalling 1296 fits
Random forest r2_score =  0.03665729974921428


Pipeline(steps=[('encoder',
                 OneHotEncoder(handle_unknown='ignore', sparse=False)),
                ('scaler', StandardScaler()),
                ('Regressor', RandomForestRegressor(bootstrap=False))])

In [5]:
"""
Adesso mi occupo di fare la parte di classificazione: voglio individuare quali siano le circoscrizioni di Trento che hanno
il più alto numero di interazioni sociali basandomi sui dati delle giornate precedenti. 
Il dataset sarà uguale a quello precedente con la differenza che la parte di conteggi delle X sarà divisa per circoscrizione
Dal momento che ci sono 12 circoscrizioni i parametri di input saranno 
[interazioni sociali giorno i-2 divisi per circoscrizione][interazioni sociali giorno i-1 divisi per circoscrizione] + altri parametri
Per quanto riguarda i target, considereremo la circoscrizione con il maggior numero di tweets.
Inizialmente provo a lavorare con un Classificatore random forest, dal momento che abbiamo pochi elementi
"""
# Random Forest Classifier
supp_ML.Random_Forest_Classifier_Circoscrizione(data)

['CENTRO STORICO PIEDICASTELLO', 'OLTREFERSINA', 'CENTRO STORICO PIEDICASTELLO', 'CENTRO STORICO PIEDICASTELLO', 'CENTRO STORICO PIEDICASTELLO', 'CENTRO STORICO PIEDICASTELLO', 'CENTRO STORICO PIEDICASTELLO', 'CENTRO STORICO PIEDICASTELLO', 'CENTRO STORICO PIEDICASTELLO', 'OLTREFERSINA', 'CENTRO STORICO PIEDICASTELLO', 'CENTRO STORICO PIEDICASTELLO', 'CENTRO STORICO PIEDICASTELLO', 'CENTRO STORICO PIEDICASTELLO', 'CENTRO STORICO PIEDICASTELLO', 'CENTRO STORICO PIEDICASTELLO', 'CENTRO STORICO PIEDICASTELLO', 'CENTRO STORICO PIEDICASTELLO', 'CENTRO STORICO PIEDICASTELLO', 'CENTRO STORICO PIEDICASTELLO', 'CENTRO STORICO PIEDICASTELLO', 'CENTRO STORICO PIEDICASTELLO', 'CENTRO STORICO PIEDICASTELLO', 'OLTREFERSINA', 'CENTRO STORICO PIEDICASTELLO', 'OLTREFERSINA', 'CENTRO STORICO PIEDICASTELLO', 'OLTREFERSINA', 'OLTREFERSINA', 'CENTRO STORICO PIEDICASTELLO', 'CENTRO STORICO PIEDICASTELLO', 'CENTRO STORICO PIEDICASTELLO', 'CENTRO STORICO PIEDICASTELLO', 'OLTREFERSINA', 'CENTRO STORICO PIEDICA